This the code for walking through the lesson examples for your reference.

## Code for Building Synthetic Dataset

In [57]:
import pandas as pd
import numpy as np

In [58]:
#build synthetic line level example
NUMBER_RECORDS = 100000
NUMBER_ENCOUNTERS = 7800
NUMBER_PATIENTS = 1000

In [59]:
dx_code_list = ["dx_code_" + str(x) for x in np.arange(1,100000)]

In [60]:
#print(dx_code_list)

In [61]:
procedure_code_list =["procedure_code_" + str(x) for x in np.arange(0,73000)]

In [62]:
#print(procedure_code_list)

In [63]:
# Create random list of code sets for diagnosis, procedure, medication, and lab codes
dx_code_list = ["dx_code_" + str(x) for x in np.arange(1,100000)]
procedure_code_list =["procedure_code_" + str(x) for x in np.arange(0,73000)]
medication_code_list = ["medication_code_" + str(x) for x in np.arange(0,10000)]
lab_code_list = ["lab_code_" + str(x) for x in np.arange(0,10000)]

In [64]:
patient_id_list = ["udacity_health_patient_id_" + str(x) for x in np.arange(1, NUMBER_PATIENTS +1)]
encounter_id_list = ["udacity_health_encounter_id_" + str(x) for x in np.arange(1, NUMBER_ENCOUNTERS +1)]

In [65]:
np.random.dirichlet((1,1,1,1), 2)

array([[0.2262601 , 0.37463069, 0.36510079, 0.03400843],
       [0.06232512, 0.24166103, 0.28239623, 0.41361762]])

In [66]:
def random_value_selection(field_value_list, number_records): #-----Define a function
    #build normal probability distribution 
    field_prob_dist = np.random.dirichlet(np.ones(len(field_value_list)), size=1)[0] 
    #build random value list for field
    field_random_values = np.random.choice(field_value_list, number_records, p=field_prob_dist)
    return field_random_values

In [67]:
#patient_values = random_value_selection(patient_id_list, NUMBER_RECORDS)
encounter_values = random_value_selection(encounter_id_list, NUMBER_RECORDS)

In [68]:
len(encounter_values)

100000

In [69]:
encounter_patient_mapping = dict(zip(encounter_id_list,   random_value_selection(patient_id_list, NUMBER_ENCOUNTERS)))

In [70]:
#encounter_patient_mapping

In [71]:
len(encounter_patient_mapping)

7800

In [72]:
encounter_patient_mapping = dict(zip(encounter_id_list,   random_value_selection(patient_id_list, NUMBER_ENCOUNTERS)))
patient_values = [encounter_patient_mapping[x] for x in encounter_values]

In [73]:
dx_value_mapping = dict(zip(encounter_id_list, random_value_selection(dx_code_list, NUMBER_ENCOUNTERS) ))
dx_values = [dx_value_mapping[x] for x in encounter_values ]

In [74]:
procedure_values = random_value_selection(procedure_code_list, NUMBER_RECORDS)
medication_values = random_value_selection(medication_code_list, NUMBER_RECORDS)
lab_values = random_value_selection(lab_code_list, NUMBER_RECORDS)

In [75]:
triplet_prob_choice = np.random.choice([0, 1, 2], NUMBER_RECORDS, p= np.random.dirichlet(np.ones(3), size=1)[0] )
line_triplet_values = list(zip(procedure_values, medication_values, lab_values, triplet_prob_choice))
selected_procedure_values = [x[0] if x[3] == 0 else np.nan for x in line_triplet_values ]
selected_medication_values = [x[1] if x[3] == 1 else np.nan for x in line_triplet_values]
selected_lab_values = [x[2] if x[3] == 2 else np.nan for x in line_triplet_values]

In [76]:
#add label
patient_label_mapping = dict(zip( patient_id_list, np.random.choice([0, 1], NUMBER_PATIENTS, replace=True, 
                                                                    p=[0.88, 0.12]) ))
label_values = [patient_label_mapping[x] for x in patient_values]

In [77]:
line_df = pd.DataFrame({ "ENCOUNTER_ID": encounter_values,
                        "PATIENT_ID": patient_values,
                        "PRINCIPAL_DIAGNOSIS_CODE": dx_values,
                        "PROCEDURE_CODE": selected_procedure_values,
                        "MEDICATION_CODE": selected_medication_values,
                        "LAB_CODE": selected_lab_values,
                        "LABEL": label_values
                       })

In [78]:
line_df

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_640,udacity_health_patient_id_530,dx_code_48194,NaN,medication_code_7186,NaN,0
1,udacity_health_encounter_id_6124,udacity_health_patient_id_600,dx_code_59221,procedure_code_5164,NaN,NaN,1
2,udacity_health_encounter_id_221,udacity_health_patient_id_754,dx_code_90138,NaN,medication_code_3993,NaN,0
3,udacity_health_encounter_id_2889,udacity_health_patient_id_10,dx_code_73652,procedure_code_27037,NaN,NaN,1
4,udacity_health_encounter_id_5845,udacity_health_patient_id_900,dx_code_50219,procedure_code_55964,NaN,NaN,0
...,...,...,...,...,...,...,...
99995,udacity_health_encounter_id_7494,udacity_health_patient_id_343,dx_code_43123,NaN,medication_code_5796,NaN,0
99996,udacity_health_encounter_id_2774,udacity_health_patient_id_377,dx_code_48559,NaN,medication_code_1773,NaN,0
99997,udacity_health_encounter_id_2461,udacity_health_patient_id_145,dx_code_48000,NaN,medication_code_7342,NaN,0
99998,udacity_health_encounter_id_7449,udacity_health_patient_id_855,dx_code_99654,NaN,medication_code_1276,NaN,0


In [79]:
#line_df.to_csv("./data/SYNTHETIC_EHR_DATASET.csv", index=False)

## 1. Converting Line to Encounter Representation

### Load Synthetic EHR Line Dataset

In [80]:
ehr_line_df = pd.read_csv("./data/SYNTHETIC_EHR_DATASET.csv")

In [81]:
ehr_line_df.head()

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1933,udacity_health_patient_id_817,dx_code_56522,procedure_code_20005,NaN,NaN,0
1,udacity_health_encounter_id_5664,udacity_health_patient_id_594,dx_code_39264,NaN,medication_code_7471,NaN,0
2,udacity_health_encounter_id_1946,udacity_health_patient_id_169,dx_code_83619,NaN,NaN,lab_code_5311,0
3,udacity_health_encounter_id_1528,udacity_health_patient_id_870,dx_code_97434,procedure_code_31111,NaN,NaN,0
4,udacity_health_encounter_id_7185,udacity_health_patient_id_722,dx_code_68924,NaN,medication_code_2250,NaN,0


In [82]:
ehr_line_df.tail()

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
99995,udacity_health_encounter_id_7533,udacity_health_patient_id_191,dx_code_92422,NaN,medication_code_5271,NaN,0
99996,udacity_health_encounter_id_5449,udacity_health_patient_id_651,dx_code_79555,NaN,NaN,lab_code_9190,0
99997,udacity_health_encounter_id_3397,udacity_health_patient_id_1000,dx_code_69775,procedure_code_30143,NaN,NaN,1
99998,udacity_health_encounter_id_2442,udacity_health_patient_id_898,dx_code_61239,NaN,medication_code_129,NaN,0
99999,udacity_health_encounter_id_5602,udacity_health_patient_id_80,dx_code_45899,NaN,NaN,lab_code_9818,1


In [83]:
ehr_line_df[ehr_line_df['ENCOUNTER_ID']=='udacity_health_encounter_id_100']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
14286,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_4198,0
19091,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_7982,NaN,0
29530,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_6603,0
34583,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_332,0
62325,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_2452,NaN,0
64967,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_1431,0
94636,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_279,NaN,0


In [84]:
ehr_line_df[ehr_line_df['PATIENT_ID']=='udacity_health_patient_id_585']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
161,udacity_health_encounter_id_983,udacity_health_patient_id_585,dx_code_27700,NaN,medication_code_5201,NaN,0
184,udacity_health_encounter_id_7155,udacity_health_patient_id_585,dx_code_75500,NaN,medication_code_6633,NaN,0
841,udacity_health_encounter_id_7155,udacity_health_patient_id_585,dx_code_75500,procedure_code_31792,NaN,NaN,0
1565,udacity_health_encounter_id_983,udacity_health_patient_id_585,dx_code_27700,NaN,medication_code_6972,NaN,0
2141,udacity_health_encounter_id_7155,udacity_health_patient_id_585,dx_code_75500,NaN,medication_code_5212,NaN,0
...,...,...,...,...,...,...,...
94636,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_279,NaN,0
96140,udacity_health_encounter_id_1970,udacity_health_patient_id_585,dx_code_75302,NaN,medication_code_8919,NaN,0
99020,udacity_health_encounter_id_6528,udacity_health_patient_id_585,dx_code_87709,NaN,medication_code_4594,NaN,0
99096,udacity_health_encounter_id_7155,udacity_health_patient_id_585,dx_code_75500,procedure_code_32548,NaN,NaN,0


In [85]:
#note that this is for illustrative purposes only and for practicing key skills, 
# the actual data representation and combinations of codes not indicative of real thing

### Convert Line to Encounter Representation

In [86]:
# grouping fields 
grouping_field_list = ['ENCOUNTER_ID', 'PATIENT_ID', 'PRINCIPAL_DIAGNOSIS_CODE']
non_grouped_field_list = [c for c in ehr_line_df.columns if c not in grouping_field_list]

In [87]:
encounter_df = ehr_line_df.groupby(grouping_field_list)[non_grouped_field_list].agg(lambda x: 
                                                        list([y for y in x if y is not np.nan ] ) ).reset_index()

In [88]:
encounter_df.head()

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,udacity_health_encounter_id_10,udacity_health_patient_id_188,dx_code_74047,[],"[medication_code_7789, medication_code_3560, m...",[],"[1, 1, 1, 1]"
2,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,[],"[medication_code_7982, medication_code_2452, m...","[lab_code_4198, lab_code_6603, lab_code_332, l...","[0, 0, 0, 0, 0, 0, 0]"
3,udacity_health_encounter_id_1000,udacity_health_patient_id_525,dx_code_61569,[],[medication_code_4036],[],[0]
4,udacity_health_encounter_id_1001,udacity_health_patient_id_950,dx_code_90172,[procedure_code_30555],"[medication_code_6755, medication_code_5045]",[lab_code_9112],"[0, 0, 0, 0]"


In [89]:
encounter_df[0:5]

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,udacity_health_encounter_id_10,udacity_health_patient_id_188,dx_code_74047,[],"[medication_code_7789, medication_code_3560, m...",[],"[1, 1, 1, 1]"
2,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,[],"[medication_code_7982, medication_code_2452, m...","[lab_code_4198, lab_code_6603, lab_code_332, l...","[0, 0, 0, 0, 0, 0, 0]"
3,udacity_health_encounter_id_1000,udacity_health_patient_id_525,dx_code_61569,[],[medication_code_4036],[],[0]
4,udacity_health_encounter_id_1001,udacity_health_patient_id_950,dx_code_90172,[procedure_code_30555],"[medication_code_6755, medication_code_5045]",[lab_code_9112],"[0, 0, 0, 0]"


In [90]:
ehr_line_df[ehr_line_df['ENCOUNTER_ID']=='udacity_health_encounter_id_1']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
3246,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_2350,NaN,0
7901,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_8630,NaN,0
11765,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,NaN,lab_code_8835,0
11950,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,procedure_code_58552,NaN,NaN,0
16057,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_4030,NaN,0
17961,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,procedure_code_39776,NaN,NaN,0
24877,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_431,NaN,0
26235,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_7435,NaN,0
35490,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,NaN,lab_code_9859,0
42855,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_4338,NaN,0


In [91]:
encounter_df[encounter_df['ENCOUNTER_ID']=='udacity_health_encounter_id_1']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [92]:
encounter_df[encounter_df['ENCOUNTER_ID']=='udacity_health_encounter_id_5']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
4142,udacity_health_encounter_id_5,udacity_health_patient_id_632,dx_code_80370,"[procedure_code_14389, procedure_code_71629]","[medication_code_2620, medication_code_3971, m...",[lab_code_9065],"[0, 0, 0, 0, 0, 0]"


## 2. Converting Encounter to Longitudinal Representation

In [93]:
encounter_df.head()

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,udacity_health_encounter_id_10,udacity_health_patient_id_188,dx_code_74047,[],"[medication_code_7789, medication_code_3560, m...",[],"[1, 1, 1, 1]"
2,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,[],"[medication_code_7982, medication_code_2452, m...","[lab_code_4198, lab_code_6603, lab_code_332, l...","[0, 0, 0, 0, 0, 0, 0]"
3,udacity_health_encounter_id_1000,udacity_health_patient_id_525,dx_code_61569,[],[medication_code_4036],[],[0]
4,udacity_health_encounter_id_1001,udacity_health_patient_id_950,dx_code_90172,[procedure_code_30555],"[medication_code_6755, medication_code_5045]",[lab_code_9112],"[0, 0, 0, 0]"


In [94]:
patient_grouping_field_list = ["PATIENT_ID"]
non_patient_agg_field_list = [c for c in encounter_df.columns if c not in patient_grouping_field_list]

In [95]:
long_df = encounter_df.groupby(patient_grouping_field_list)[non_patient_agg_field_list].agg(lambda x: 
                                                        list([y for y in x if y is not np.nan ] ) ).reset_index()

In [96]:
long_df.head()

,PATIENT_ID,ENCOUNTER_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_patient_id_1,"[udacity_health_encounter_id_1038, udacity_hea...","[dx_code_36196, dx_code_63471, dx_code_29114, ...","[[procedure_code_36285, procedure_code_21124, ...","[[medication_code_3772, medication_code_9214, ...","[[lab_code_3982, lab_code_306], [], [lab_code_...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,udacity_health_patient_id_10,"[udacity_health_encounter_id_1110, udacity_hea...","[dx_code_29609, dx_code_268, dx_code_26932, dx...","[[procedure_code_9379], [procedure_code_3052, ...","[[medication_code_7371, medication_code_2104, ...","[[lab_code_6457], [lab_code_2180, lab_code_693...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ..."
2,udacity_health_patient_id_100,"[udacity_health_encounter_id_1205, udacity_hea...","[dx_code_32095, dx_code_45376, dx_code_48998, ...","[[procedure_code_71055, procedure_code_29744, ...","[[medication_code_2399, medication_code_966, m...","[[lab_code_4928, lab_code_6524, lab_code_1713,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,udacity_health_patient_id_1000,"[udacity_health_encounter_id_1105, udacity_hea...","[dx_code_53764, dx_code_50924, dx_code_80218, ...","[[], [procedure_code_7870, procedure_code_2169...","[[medication_code_6580, medication_code_3007],...","[[], [lab_code_5468], [lab_code_7607], [lab_co...","[[1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1..."
4,udacity_health_patient_id_101,"[udacity_health_encounter_id_2058, udacity_hea...","[dx_code_13590, dx_code_29551]","[[procedure_code_45319, procedure_code_67294],...","[[medication_code_2532], [medication_code_4311...","[[lab_code_1206, lab_code_9967], []]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]"


In [97]:
example_patient_history = long_df[long_df['PATIENT_ID']=='udacity_health_patient_id_310']

In [98]:
example_patient_history

,PATIENT_ID,ENCOUNTER_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
211,udacity_health_patient_id_310,"[udacity_health_encounter_id_4647, udacity_hea...","[dx_code_74153, dx_code_95836, dx_code_39465, ...","[[procedure_code_40521, procedure_code_52188, ...","[[medication_code_7251, medication_code_2765, ...","[[lab_code_1372], [lab_code_1794], [lab_code_1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [99]:
list(example_patient_history['ENCOUNTER_ID'].values)

[['udacity_health_encounter_id_4647',
  'udacity_health_encounter_id_5010',
  'udacity_health_encounter_id_551',
  'udacity_health_encounter_id_7210',
  'udacity_health_encounter_id_7331']]

In [100]:
list(example_patient_history['PRINCIPAL_DIAGNOSIS_CODE'].values)

[['dx_code_74153',
  'dx_code_95836',
  'dx_code_39465',
  'dx_code_66358',
  'dx_code_99743']]

In [101]:
list(example_patient_history['PATIENT_ID'].values)

['udacity_health_patient_id_310']

In [102]:
list(example_patient_history['PROCEDURE_CODE'].values)

[[['procedure_code_40521',
   'procedure_code_52188',
   'procedure_code_57020',
   'procedure_code_11784',
   'procedure_code_50478'],
  ['procedure_code_12696',
   'procedure_code_2873',
   'procedure_code_28392',
   'procedure_code_1398'],
  ['procedure_code_49962'],
  [],
  ['procedure_code_31171']]]

## 3. How to Split Dataset at Patient Level

#### ***Objective:*** 
- Split dataset at patient level into train and test partitions
- Validate that split was done correctly

#### Dataset Splitting Tests
- Patient data in only one partition
- Total unique number of patients across all partitions = total number unique patients in original full dataset
- Total number of rows original dataset = sum of rows across splits

In [103]:
PATIENT_ID_FIELD = 'PATIENT_ID'
TEST_PERCENTAGE = 0.2

In [104]:
def split_dataset_patient_level(df, key, test_percentage=0.2):
    df = df.iloc[np.random.permutation(len(df))]
    unique_values = df[key].unique()
    total_values = len(unique_values)
    sample_size = round(total_values * (1 - test_percentage ))
    train = df[df[key].isin(unique_values[:sample_size])].reset_index(drop=True)
    test = df[df[key].isin(unique_values[sample_size:])].reset_index(drop=True)
    return train, test

In [105]:
train_df.head()

NameError: name 'train_df' is not defined

In [ ]:
test_df.head()

In [ ]:
train_df, test_df = split_dataset_patient_level(encounter_df, PATIENT_ID_FIELD, TEST_PERCENTAGE)

In [ ]:
assert len(set(train_df[PATIENT_ID_FIELD].unique()).intersection(set(test_df[PATIENT_ID_FIELD].unique()))) == 0
print("Test passed for patient data in only one partition")

In [ ]:
assert (train_df[PATIENT_ID_FIELD].nunique()  + test_df[PATIENT_ID_FIELD].nunique()) == encounter_df[PATIENT_ID_FIELD].nunique()
print("Test passed for number of unique patients being equal!")

In [ ]:
assert len(train_df)  + len(test_df) == len(encounter_df)
print("Test passed for number of total rows equal!")

## 4. ETL with TF Dataset API and Pandas

NOTE: In some cases you may need to preprocess Pandas Dataframe to removed mixed types. In particular, remove null values and impute or remove rows (we will later impute with zero for numerical features).

In [ ]:
import tensorflow as tf

In [ ]:
swiss_dataset_path = "./data/processed_swiss.csv"
swiss_df = pd.read_csv(swiss_dataset_path)
selected_col_list = ['age', 'thalach', 'cp', 'num_label']
subset_swiss_df = swiss_df[selected_col_list]

In [ ]:
swiss_df.head()

In [ ]:
subset_swiss_df.head()

In [ ]:
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
print(sample_tf_ds)

In [ ]:
BATCH_SIZE = 64
PREDICTOR_FIELD = 'num_label'
sample_tf_ds = df_to_dataset(subset_swiss_df, PREDICTOR_FIELD, batch_size=BATCH_SIZE)

In [ ]:
sample_feature_batch = next(iter(sample_tf_ds))[0]
sample_feature_batch

In [ ]:
sample_label_batch = next(iter(sample_tf_ds))[1]
sample_label_batch

## 5. Building Numerical Feature with TF Feature Column API

In [ ]:
subset_swiss_df.head()

In [ ]:
age_mean = subset_swiss_df['age'].describe()['mean']
age_std = subset_swiss_df['age'].describe()['std']
print("Mean age:{}\nStandard Deviation Age:{}".format(age_mean, age_std))

In [ ]:
import functools
def normalize_numeric_with_zscore(col, mean, std):
    return (col - mean)/std

def create_tf_numeric_feature(col, MEAN, STD,   default_value=0):
    normalizer = functools.partial(normalize_numeric_with_zscore, mean=MEAN, std=STD)
    return tf.feature_column.numeric_column(
    key=col, default_value = default_value, normalizer_fn=normalizer, dtype=tf.float64)

In [ ]:
age_tf_feature = create_tf_numeric_feature('age', age_mean, age_std)

In [ ]:
def demo(feature_column, example_batch):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch))

In [ ]:
print("Example continuous field:\n{}\n".format(age_tf_feature))
demo(age_tf_feature, sample_feature_batch)

## 6. Building Categorical Features with TF Feature Column API

In [ ]:
categorical_example_df = encounter_df[['ENCOUNTER_ID', 'PRINCIPAL_DIAGNOSIS_CODE', 'LABEL']] 

In [ ]:
# for this task need to convert label from array to scalar value
categorical_example_df['LABEL'] = categorical_example_df['LABEL'].apply(lambda x: np.unique(x)[0])

In [ ]:
categorical_example_df.head()

### High Cardinality for Principal Diagnosis Code

In [ ]:
categorical_example_df['PRINCIPAL_DIAGNOSIS_CODE'].nunique()

### Generate Vocabulary File

In [ ]:
#make vocab dir
import os
#os.mkdir("./vocab/")

In [ ]:
# build vocab for categorical features
def write_vocabulary_file(vocab_list, field_name, default_value, vocab_dir='./vocab/'):
    output_file_path = os.path.join(vocab_dir, str(field_name) + "_vocab.txt")
    # put default value in first row as TF requires
    vocab_list = np.insert(vocab_list, 0, default_value, axis=0) 
    df = pd.DataFrame(vocab_list).to_csv(output_file_path, index=None, header=None)
    return output_file_path

def build_vocab_files(df, categorical_column_list, default_value='00'):
    vocab_files_list = []
    for c in categorical_column_list:
        v_file = write_vocabulary_file(df[c].unique(), c, default_value)
        vocab_files_list.append(v_file)
    return vocab_files_list

In [ ]:
categorical_field_list = ["PRINCIPAL_DIAGNOSIS_CODE"]
vocab_files_list = build_vocab_files(categorical_example_df, categorical_field_list)

### Build TF Dataset from Pandas Dataframe

In [87]:
BATCH_SIZE = 64
PREDICTOR_FIELD = 'LABEL'
categorical_tf_ds = df_to_dataset(categorical_example_df, PREDICTOR_FIELD, batch_size=BATCH_SIZE)

### Use TF Feature Column API to read from vocab file

In [88]:
vocab_files_list[0]

'./vocab/PRINCIPAL_DIAGNOSIS_CODE_vocab.txt'

In [89]:
principal_diagnosis_vocab = tf.feature_column.categorical_column_with_vocabulary_file(
            key="PRINCIPAL_DIAGNOSIS_CODE", vocabulary_file = vocab_files_list[0], num_oov_buckets=1)

INFO:tensorflow:vocabulary_size = 6753 in PRINCIPAL_DIAGNOSIS_CODE is inferred from the number of elements in the vocabulary_file ./vocab/PRINCIPAL_DIAGNOSIS_CODE_vocab.txt.


### Create one-hot encoding  from vocab column feature function

In [90]:
one_hot_principal_diagnosis_feature = tf.feature_column.indicator_column(principal_diagnosis_vocab)

In [91]:
categorical_tf_ds_batch = next(iter(categorical_tf_ds))[0]

In [92]:
demo(one_hot_principal_diagnosis_feature, categorical_tf_ds_batch)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 6754), dtype=float32)
